# Nested Objects and Lists

Kor attempts to make it easy to capture more complex structure during extraction.

**ATTENTION** At the moment to use either nested objects or nested lists, one should use the `json` encoder.

In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.insert(0, "../../")

In [2]:
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

In [3]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    max_tokens=2000,
    frequency_penalty=0,
    presence_penalty=0,
    top_p=1.0,
)

WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.
WARNING! presence_penalty is not default parameter.
                    presence_penalty was transferred to model_kwargs.
                    Please confirm that presence_penalty is what you intended.
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


## Nested Objects

Here, we'll introduce an `Address` object which will be neste inside of the main schema.

In [4]:
from_address = Object(
    id="from_address",
    description="Person moved away from this address",
    attributes=[
        Text(id="street"),
        Text(id="city"),
        Text(id="state"),
        Text(id="zipcode"),
        Text(id="country", description="A country in the world; e.g., France."),
    ],
    examples=[
        (
            "100 Main St, Boston, MA, 23232, USA",
            {
                "street": "100 Marlo St",
                "city": "Boston",
                "state": "MA",
                "zipcode": "23232",
                "country": "USA",
            },
        )
    ],
)

to_address = from_address.replace(
    id="to_address", description="Address to which the person is moving"
)

schema = Object(
    id="information",
    attributes=[
        Text(
            id="person_name",
            description="The full name of the person or partial name",
            examples=[("John Smith was here", "John Smith")],
        ),
        from_address,
        to_address,
    ],
    many=True,
)

## JSON encoding

To use nested objects, at least for now we have to swap to the JSON encoder.

Anecdotally, CSV encoding seems to produce more robust extraction results, so JSON encoding may perform worse even though it's more flexible.

In [5]:
chain = create_extraction_chain(
    llm, schema, encoder_or_encoder_class="json", input_formatter=None
)

In [6]:
chain.run(
    "Alice Doe moved from New York to Boston, MA while Bob Smith did the opposite."
)["data"]

{'information': [{'person_name': 'Alice Doe',
   'to_address': {'street': '100 Main St',
    'city': 'Boston',
    'state': 'MA',
    'zipcode': '23232',
    'country': 'USA'}},
  {'person_name': 'Bob Smith',
   'to_address': {'street': '100 Main St',
    'city': 'New York',
    'state': 'NY',
    'zipcode': '10001',
    'country': 'USA'}}]}

In [7]:
chain.run(
    "Alice Doe and Bob Smith moved from New York to Boston. Andrew was 12 years"
    " old. He also moved to Boston. So did Joana and Paul. Betty did the opposite."
)["data"]

{'information': [{'person_name': 'Alice Doe',
   'to_address': {'city': 'Boston'}},
  {'person_name': 'Bob Smith', 'to_address': {'city': 'Boston'}},
  {'person_name': 'Andrew', 'to_address': {'city': 'Boston'}},
  {'person_name': 'Joana', 'to_address': {'city': 'Boston'}},
  {'person_name': 'Paul', 'to_address': {'city': 'Boston'}}]}

## Nested Lists

Let's repeat the same schema as above, but let the address be a `many=True` field.

In [8]:
from_address = Object(
    id="from_address",
    description="Person moved away from this address",
    attributes=[
        Text(id="street"),
        Text(id="city"),
        Text(id="state"),
        Text(id="zipcode"),
        Text(id="country", description="A country in the world; e.g., France."),
    ],
    examples=[
        (
            "100 Main St, Boston,MA, 23232, USA",
            {
                "street": "100 Marlo St",
                "city": "Boston",
                "state": "MA",
                "zipcode": "23232",
                "country": "USA",
            },
        )
    ],
    many=True,  # <-- PLEASE NOTE THIS CHANGE
)

to_address = from_address.replace(
    id="to_address", description="Address to which the person is moving"
)

schema = Object(
    id="information",
    attributes=[
        Text(
            id="person_name",
            description="The full name of the person or partial name",
            examples=[("John Smith was here", "John Smith")],
        ),
        from_address,
        to_address,
    ],
    many=True,
)

In [9]:
chain = create_extraction_chain(llm, schema, encoder_or_encoder_class="json")

In [10]:
chain.run(
    "Alice Doe and Bob Smith moved from New York to Boston. Bob later moved to LA."
)["data"]

{'information': [{'person_name': 'Alice Doe',
   'to_address': [{'street': 'New York'}]},
  {'person_name': 'Bob Smith', 'to_address': [{'street': 'New York'}]},
  {'person_name': 'Bob Smith', 'to_address': [{'street': 'Boston'}]},
  {'person_name': 'Bob Smith', 'to_address': [{'street': 'LA'}]}]}